In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cee-498-project1-london-bike-sharing/train.csv
/kaggle/input/cee-498-project1-london-bike-sharing/sample.csv
/kaggle/input/cee-498-project1-london-bike-sharing/test.csv


# Kaggle Competition

The goal of this project is to predict the ranges of cnt (the count of a new bike shares) each hour based on the given factors.

# Variables Preparation

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import sklearn.metrics

import matplotlib.pyplot as plt
%matplotlib inline
import datetime

In [3]:
train_df = pd.read_csv("../input/cee-498-project1-london-bike-sharing/train.csv")
test_df = pd.read_csv("../input/cee-498-project1-london-bike-sharing/test.csv")

In [4]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])

train_df['hour'] = train_df['timestamp'].dt.hour
train_df['month'] = train_df['timestamp'].dt.month
train_df['day'] = train_df['timestamp'].dt.day
train_df['weekday'] = train_df['timestamp'].dt.dayofweek
train_df['year'] = train_df['timestamp'].dt.year

train_df.t1 = train_df.t1.astype(int)
train_df.t2 = train_df.t2.astype(int)
train_df.hum = train_df.hum.astype(int)
train_df.wind_speed = train_df.wind_speed.astype(int)
train_df.weather_code = train_df.weather_code.astype(int)
train_df.is_holiday = train_df.is_holiday.astype(int)
train_df.is_weekend = train_df.is_weekend.astype(int)
train_df.season = train_df.season.astype(int)

train_df.drop('timestamp', axis = 1, inplace = True)
train_df.drop(['t2'], axis = 1)

,cnt,t1,hum,wind_speed,weather_code,is_holiday,is_weekend,season,hour,month,day,weekday,year
0,134,2,96,0,1,0,1,3,2,1,4,6,2015
1,75,1,100,7,4,0,1,3,7,1,4,6,2015
2,131,1,96,8,4,0,1,3,8,1,4,6,2015
3,301,2,100,9,3,0,1,3,9,1,4,6,2015
4,528,3,93,12,3,0,1,3,10,1,4,6,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12218,1042,5,81,19,3,0,0,3,19,1,3,1,2017
12219,541,5,81,21,4,0,0,3,20,1,3,1,2017
12220,337,5,78,24,4,0,0,3,21,1,3,1,2017
12221,224,5,76,23,4,0,0,3,22,1,3,1,2017


In [5]:
sample = pd.read_csv("../input/cee-498-project1-london-bike-sharing/test.csv")
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])

test_df['hour'] = test_df['timestamp'].dt.hour
test_df['month'] = test_df['timestamp'].dt.month
test_df['day'] = test_df['timestamp'].dt.day
test_df['weekday'] = test_df['timestamp'].dt.dayofweek
test_df['year'] = test_df['timestamp'].dt.year

test_df.drop(['timestamp'], axis = 1, inplace = True)

test_df.t1 = test_df.t1.astype(int)
test_df.t2 = test_df.t2.astype(int)
test_df.hum = test_df.hum.astype(int)
test_df.wind_speed = test_df.wind_speed.astype(int)
test_df.weather_code = test_df.weather_code.astype(int)
test_df.is_holiday = test_df.is_holiday.astype(int)
test_df.is_weekend = test_df.is_weekend.astype(int)
test_df.season = test_df.season.astype(int)

test_df = test_df.drop(['t2'], axis =1)
test_df


,t1,hum,wind_speed,weather_code,is_holiday,is_weekend,season,hour,month,day,weekday,year
0,3,93,6,3,0,1,3,0,1,4,6,2015
1,3,93,5,1,0,1,3,1,1,4,6,2015
2,2,100,0,1,0,1,3,3,1,4,6,2015
3,2,93,6,1,0,1,3,4,1,4,6,2015
4,2,93,4,1,0,1,3,5,1,4,6,2015
...,...,...,...,...,...,...,...,...,...,...,...,...
5186,5,76,11,1,1,0,3,16,1,2,0,2017
5187,3,81,11,1,1,0,3,19,1,2,0,2017
5188,2,81,14,1,0,0,3,9,1,3,1,2017
5189,4,78,21,1,0,0,3,11,1,3,1,2017


In [6]:
y = train_df['cnt']
x = train_df.drop(['cnt','t2'], axis =1)

# Regular Neural Network

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size=50).repeat()

model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(units=20, input_shape=(12,)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=10, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=1))

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr = 0.001),loss = 'mean_squared_error',
    metrics = [tf.keras.metrics.RootMeanSquaredError()]
)

def scheduler(epoch, lr):
    return lr * (0.9 ** (epoch // 100))
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

model.fit(x, y, batch_size=100, epochs=300, validation_split=0, callbacks=[callback])

Epoch 1/300
123/123 [==============================] - 0s 3ms/step - loss: 2454477.7500 - root_mean_squared_error: 1566.6772
Epoch 2/300
123/123 [==============================] - 0s 4ms/step - loss: 2447483.7500 - root_mean_squared_error: 1564.4436
Epoch 3/300
123/123 [==============================] - 0s 4ms/step - loss: 2438447.5000 - root_mean_squared_error: 1561.5529
Epoch 4/300
123/123 [==============================] - 0s 4ms/step - loss: 2426883.7500 - root_mean_squared_error: 1557.8458
Epoch 5/300
123/123 [==============================] - 0s 4ms/step - loss: 2412609.7500 - root_mean_squared_error: 1553.2578
Epoch 6/300
123/123 [==============================] - 0s 4ms/step - loss: 2395587.0000 - root_mean_squared_error: 1547.7684
Epoch 7/300
123/123 [==============================] - 0s 4ms/step - loss: 2374748.5000 - root_mean_squared_error: 1541.0219
Epoch 8/300
123/123 [==============================] - 0s 4ms/step - loss: 2351074.2500 - root_mean_squared_error: 1533.3213


# Test Data

In [8]:
Prediction = model.predict(test_df)

In [9]:
sample['cnt'] = Prediction
result = sample.drop(['t1','t2','hum','wind_speed','weather_code','is_holiday','is_weekend','season'],axis = 1)
result

,timestamp,cnt
0,2015-01-04 00:00:00,302.579285
1,2015-01-04 01:00:00,297.230621
2,2015-01-04 03:00:00,267.549133
3,2015-01-04 04:00:00,206.531586
4,2015-01-04 05:00:00,162.121216
...,...,...
5186,2017-01-02 16:00:00,1281.073853
5187,2017-01-02 19:00:00,452.623596
5188,2017-01-03 09:00:00,1933.217041
5189,2017-01-03 11:00:00,738.997742


In [10]:
result.to_csv('Result_Anye Wang.csv', index = False)